# Introduction

This notebook is designed to generate the necessary databases for studying the high-voltage (HV) motors as in [Jerome, A., Ljunggren, M. and Janssen, M. (2023) Is repair of energy using products environmentally beneficial? The case of high voltage electric motors](https://doi.org/10.1016/j.resconrec.2023.107038) (*Resources Conservation and Recycling*, 196, 107038).

**References**

Chordia, M., Nordelöf, A., Ellingsen, L.A.W., 2021. Environmental life cycle implications of upscaling lithium-ion battery production. International Journal of Life Cycle Assessment 26, 2024–2039. [DOI link](https://doi.org/10.1007/s11367-021-01976-0)

Nordelöf, A., Grunditz, E., Tillman, A.-M., Thiringer, T., Alatalo, M., 2017a. A Scalable Life Cycle Inventory of an Electrical Automotive Traction Machine. Gothenburg, Sweden. [Link](https://research.chalmers.se/en/publication/248692)

Nordelöf, A., Grunditz, E., Tillman, A.-M., Thiringer, T., Alatalo, M., 2017b. Scalable PMSM LCI Model v1.01. [Link](http://cpmdatabase.cpm.chalmers.se/Scripts/sheet.asp?ActId=JT-2016-06-21-39)

Westberg, E., 2021. Environmental Impact of an Electric Motor and Drive - Life Cycle Assessment and a study of a Circular Business Model. Linköping University. [Link](http://www.diva-portal.org/smash/record.jsf?pid=diva2%3A1658729&dswid=5915)

# Library import

In [1]:
#brightway
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

#others
import pandas as pd

#own functions
from utilities_bw2 import *

# Project setup and database import

## Project setup

In [2]:
bd.projects

Brightway2 projects manager with 16 objects:
	ABB case
	Electric motors
	Motor case
	New motor case
	WT01
	ab_demo
	ab_demo_solution
	advlca24
	brightway2_test
	brightway_demo
	default
	lca_algebraic_handbook
	lifetime_test
	motor_case
	pLCA course
	test_240410
Use `projects.report()` to get a report on all projects.

In [3]:
bd.projects.set_current("lifetime_test")

In [4]:
bd.databases

Databases dictionary with 8 object(s):
	biosphere3
	ecoinvent_391_cutoff
	eol_treatment
	furniture_production
	mattress_production
	motor_production
	packaging
	primary_production

## Import of biosphere database

In [5]:
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [6]:
biosphere_db = bd.Database("biosphere3")

## Import ecoinvent

This project uses ecoinvent 3.9.1 cutoff.

In [7]:
path_datasets = "C:/Users/user_name/Downloads/ecoinvent_3.9.1_cutoff_ecoSpold02/datasets"

In [8]:
ei_db_name = "ecoinvent_391_cutoff"

In [9]:
if ei_db_name in bd.databases:
    print("Database has already been imported.")
else:
    ei_importer = bi.SingleOutputEcospold2Importer(path_datasets, ei_db_name)
    ei_importer.apply_strategies()
    ei_importer.statistics() 
    ei_importer.write_database()

Database has already been imported.


In [10]:
ei_db = bd.Database(ei_db_name)

# End-of-life treatment of materials

## Selection of relevant ecoinvent activities

In [11]:
act_eol_copper = ei_db.search("treatment of copper scrap by electrolytic refining RER")[0]

In [12]:
act_eol_steel_low_alloyed = ei_db.search("steel production, electric, low-alloyed Europe without Switzerland and Austria")[0]

## Treatment of steel

This is to create an activity which takes iron scrap in input and produces low-alloyed steel. This is the last step of steel recycling, where scraps are transformed into usable material. The activity is based on the ecoinvent dataset for steel production (steel production, electric, low-alloyed). The iron scrap that is originally in input and linked to another ecoinvent activity is removed and transferred to a negative output of the activity.

In [13]:
def create_treatment_iron_scrap(name_eol_db):
    data_eol_activity = {
        "database": name_eol_db, 
        "name": "treatment of iron scrap by steel production, low-alloyed",
        "reference product": "iron scrap, sorted, pressed",
    }
        
    act_eol_iron = act_eol_steel_low_alloyed.copy(**data_eol_activity)

    # Capture amount of iron treated and delete exchange (set to 0)
    for exc in act_eol_iron.exchanges():
        if exc["name"] == "iron scrap, sorted, pressed":
            amount_iron_scrap = exc["amount"]
            exc.delete()
    act_eol_iron.save()

    # Replace output amount to -amount_iron_scrap
    for exc in act_eol_iron.exchanges():
        if exc["type"] == "production":
            exc["amount"] = -amount_iron_scrap
            exc.save()
            act_eol_iron["production amount"] = -amount_iron_scrap
    act_eol_iron.save()

## Treatment of copper

This is to create an activity which takes copper scrap in input and produces copper, cathode. This is the last step of copper recycling, where scraps are transformed into usable material. The activity is based on the ecoinvent dataset for copper production (treatment of copper scrap by electrolytic refining). The copper scrap that is originally in input and linked to another ecoinvent activity is removed and transferred to a negative output of the activity.

In [14]:
def create_treatment_copper_scrap(name_eol_db):
    data_eol_activity = {
        "database": name_eol_db, 
        "name": "treatment of copper scrap by electrolytic refining",
        "reference product": "copper scrap, sorted, pressed",
    }
        
    act_eol_copper_new = act_eol_copper.copy(**data_eol_activity)

    # Capture amount of copper treated and delete exchange
    for exc in act_eol_copper_new.exchanges():
        if exc["name"] == "copper scrap, sorted, pressed":
            amount_copper_scrap = exc["amount"]
            exc.delete()
    act_eol_copper_new.save()

    # Replace output amount to -amount_copper_scrap
    for exc in act_eol_copper_new.exchanges():
        if exc["type"] == "production":
            exc["amount"] = -amount_copper_scrap
            exc.save()
            act_eol_copper_new["production amount"] = -amount_copper_scrap
    act_eol_copper_new.save()

## Database creation

In [15]:
def create_eol_treatment_db(name_eol_db):
    reset_brightway_database(name_eol_db)
    eol_db = bd.Database(name_eol_db)
    eol_db.register()
    create_treatment_iron_scrap(name_eol_db)
    create_treatment_copper_scrap(name_eol_db)

In [16]:
name_eol_db="eol_treatment"
create_eol_treatment_db(name_eol_db)

In [17]:
eol_db = bd.Database(name_eol_db)

# LCI

## Selection of relevant processes

### Biosphere flows

In [18]:
ef_co2 = biosphere_db.search("Carbon dioxide, fossil ('air',)")[0]

In [19]:
ef_ni_oxide = biosphere_db.search("Nitrogen oxides ('air',)")[0]

In [20]:
ef_su_dioxide = biosphere_db.search("Sulfur dioxide ('air',)")[0]

### Ecoinvent activities for material production

In [21]:
act_steel_unalloyed = ei_db.search("steel production, converter, unalloyed RER")[0]

In [22]:
act_steel_low_alloyed = ei_db.search("market for steel, low-alloyed GLO")[0]

In [23]:
act_displaced_steel_low_alloyed = ei_db.search("steel production, converter, low-alloyed RER")[0]

In [24]:
act_hot_rolling = ei_db.search("market for hot rolling, steel GLO")[0]

In [25]:
act_aluminium = ei_db.search("market for aluminium, cast alloy GLO")[0]

In [26]:
act_copper = ei_db.search("market for copper, cathode GLO")[0]

In [27]:
act_brass_market = ei_db.search("market for brass RoW")[0]

In [28]:
act_brazing = ei_db.search("brazing solder production, cadmium free RER")[0]

In [29]:
act_elec_mv = ei_db.search("market for electricity, medium voltage SE")[0]

In [30]:
act_ferrosilicon = ei_db.search("market for ferrosilicon GLO")[0]

In [31]:
act_elec_lv = ei_db.search("market for electricity, low voltage SE")[0]

In [32]:
act_propane = ei_db.search("market for liquefied petroleum gas Europe")[0]

In [33]:
act_oil = ei_db.search("market for lubricating oil RER")[0]

In [34]:
act_resin = ei_db.search("market for phenolic resin RER")[0]

In [35]:
act_quicklime = ei_db.search("market for quicklime, milled, packed RER")[0]

In [36]:
act_sulfuric_acid = ei_db.search("market for sulfuric acid RER")[0]

In [37]:
act_glass_fibre = ei_db.search("market for glass fibre GLO")[0]

In [38]:
act_sand = ei_db.search("market for silica sand GLO")[0]

In [39]:
act_silicone = ei_db.search("market for silicone product RER")[0]

In [40]:
act_epoxy_resin = ei_db.search("market for epoxy resin, liquid RER")[0]

In [41]:
act_nylon = ei_db.search("market for nylon 6 RER")[0]

In [42]:
act_heat = ei_db.search("heat, from municipal waste incineration to generic market for heat district or industrial, other than natural gas SE")[0]

In [43]:
act_incineration_copper = ei_db.search("treatment of copper in car shredder residue, municipal incineration with fly ash extraction CH")[0]

In [44]:
act_incineration_plastic = ei_db.search("treatment of waste plastic, mixture, municipal incineration with fly ash extraction CH")[0]

### End-of-life treatment activities from ecoinvent

In [45]:
act_treatment_steel = eol_db.search("treatment of iron scrap by steel production, low-alloyed")[0]

In [46]:
act_treatment_copper = eol_db.search("treatment of copper scrap by electrolytic refining")[0]

## Electrical steel sheet production

Electrical steel sheet production process starts with the production of silicon steel, made from an alloy of carbon steel with silicon and aluminium. The silicon steel is then further processed into coils by hot rolling, which are then pickled, and finally cold rolled into sheets of the desired thickness.

### Silicon steel production

Data for the inputs in the process are from Table 27 in Nordelöf et al. (2017a). The quantities are for the production of 1 kg of silicon steel.

In [47]:
si_steel_amount_steel = 0.979 #kg
si_steel_amount_ferrosilicon = 0.027 #kg
si_steel_amount_al = 0.004 #kg

Hot rolling is also added to form the coils.

In [48]:
def create_silicon_seel_production_activity(motor_db):
    create_new_activity(
        act_name= "silicon steel hot rolled production",
        unit= "kilogram",
        location = "RER",
        exchanges=[
            (act_steel_unalloyed, si_steel_amount_steel),
            (act_ferrosilicon, si_steel_amount_ferrosilicon),
            (act_aluminium, si_steel_amount_al),
            (act_hot_rolling, 1)
        ],
        database= motor_db,
        name_product= "silicon steel, hot rolled"
    )

### Electrical steel sheet production

Data for the inputs, emissions and waste in the process are from Tables 28 and 29 in Nordelöf et al. (2017a). The quantities are for the production of 1 kg of electrical steel sheets.

*NB: In Table 29 of Nordelöf et al. (2017a), emissions of sulfur oxides are reported but linked to the elementary flow "sulphur dioxide" of ecoinvent 3. Therefore, an elementary flow of sulphur dioxide is chosen.*

In [49]:
el_steel_amount_si_steel = 1.14 #kg
el_steel_amount_resin = 0.001 #kg
el_steel_amount_elec = 0.630 #kWh
el_steel_amount_propane = 0.012 #kg
el_steel_amount_sulfuric_acid = 0.019 #kg
el_steel_amount_oil = 0.4 / 1000 #kg
el_steel_amount_quicklime = 0.8 / 1000 #kg

In [50]:
el_steel_amount_co2 = 0.036 #kg
el_steel_amount_ni_oxide = 0.1 / 1000 #kg
el_steel_amount_su_oxide = 0.000000060 #kg (rounding problem if 0.06/1000/1000)
el_steel_amount_steel_scrap = 0.114 #kg
el_steel_amount_sludge = 0.0033 # kg

Sludge treatment is not included. It is assumed that the contribution of this process is negligibled in the impact of the motor production.

In [51]:
def create_electrical_seel_production_activity(motor_db):
    act_si_steel = motor_db.search("silicon steel hot rolled production RER")[0]
    create_new_activity(
        act_name= "electrical steel sheet production",
        unit= "kilogram",
        location = "SE",
        exchanges=[
            (act_si_steel, el_steel_amount_si_steel),
            (act_resin, el_steel_amount_resin),
            (act_elec_lv, el_steel_amount_elec),
            (act_propane, el_steel_amount_propane),
            (act_sulfuric_acid, el_steel_amount_sulfuric_acid),
            (act_oil, el_steel_amount_oil),
            (act_quicklime, el_steel_amount_quicklime),
            (act_treatment_steel, -round(el_steel_amount_si_steel - 1, 2)), #Use of round function to avoid getting 0.13999999
        ],
        database= motor_db,
        biosphere_exchanges=[
            (ef_co2, el_steel_amount_co2),
            (ef_ni_oxide, el_steel_amount_ni_oxide),
            (ef_su_dioxide, el_steel_amount_su_oxide)
        ],
        name_product="electrical steel sheet"
    )

## Mica tape production

Mica tape is used for insulation in the induction motor. As in Nordelöf et al. (2017a), the processing for mica tape production beyond the material constituents has not been included in the modelling for simplicity and given the very low contribution of motor insulation to the material content and the results. Mica tape is made of glass fibre, silica sand (mica) and silicone (Nordelöf et al., 2017a). Proportions are retrieved from the scalable inventory from Nordelöf et al. (2017b) (in “1. Mass configuration”, cells L22 to L24).

In [52]:
mica_tape_amount_glass_fibre = 0.19
mica_tape_amount_sand = 0.7
mica_tape_amount_silicone = 0.11

In [53]:
def create_mica_tape_production_activity(motor_db):
    create_new_activity(
        act_name= "mica tape production",
        unit= "kilogram",
        location = "RER",
        exchanges=[
            (act_glass_fibre, mica_tape_amount_glass_fibre),
            (act_sand, mica_tape_amount_sand),
            (act_silicone, mica_tape_amount_silicone)
        ],
        database= motor_db,
        name_product= "mica tape"
    )

## Motor manufacturing

No emissions are directly emitted at the production facilities. No auxiliary material has been estimated to contribute significantly to the impact of motor manufacturing. Quantities of material inputs have been estimated based on the bill of material and the material quantities ordered for manufacturing provided by the manufacturer, and they have been used to model the processes in motor manufacturing. The inventory does not include materials with a minor share in the bill of materials such as resin for the resin impregnation of the stator and paint. It has been estimated in previous studies that their impact is minor in the life cycle of the motor (Nordelöf et al., 2017a; Westberg, 2021).

Additional inputs and outputs are energy inputs and metal scraps sent to waste treatment. For simplicity, the inventory of energy use and scrap generation has been limited to their respective main contributing processes. Two processes have a high scrap rate: the punching of electrical steel sheets and the machining of the shaft. And the three most energy-consuming processes are the final testing of the motor, the vacuum-pressing resin impregnation of the stator and the machining of the rotor shaft. The other processes have been estimated to use little energy compared to the three listed above based on discussions with experts and the inventory from Nordelöf et al. (2017a). Furthermore, results from a preliminary motor manufacturing modelling based on an average energy use for the total production show that energy use during manufacturing does not have a significant impact so the omission of energy use for other processes has very little consequence on the validity of the results.

### Machinig of the shaft and pole tips

Except for motor testing and resin impregnation, energy use for specific processes is not monitored at the manufacturing facilities. For the energy use of shaft machining, data for a similar process for low-voltage permanent magnet synchronous motors, described by Nordelöf et al. (2017a), are used (Table 47 in Nordelöf et al. (2017a)). The scrap ratios are provided by the manufacturer and differ between a shaft (for IM and SM) and pole tips (for SM only).

In [54]:
shaft_machining_elec = 0.108 #kWh per kg of shaft
shaft_machining_efficiency = 0.9 #Ratio output/input
pole_tips_machining_efficiency = 0.62 #Ratio output/input

The activity includes the electricity use and recycling of scrap for the machining of 1kg of shaft or pole tips.

In [55]:
def create_shaft_machining_activity(motor_db):
    shaft_machining_scrap = 1 / shaft_machining_efficiency - 1
    create_new_activity(
        act_name= "shaft machining",
        unit= "kilogram",
        location = "SE",
        exchanges=[
            (act_elec_mv, shaft_machining_elec),
            (act_treatment_steel, -shaft_machining_scrap)
        ],
        database= motor_db,
        name_product= "machined shaft"
    )

In [56]:
def create_pole_tips_machining_activity(motor_db):
    pole_tips_machining_scrap = 1 / pole_tips_machining_efficiency - 1
    create_new_activity(
        act_name= "pole tips machining",
        unit= "kilogram",
        location = "SE",
        exchanges=[
            (act_elec_mv, shaft_machining_elec),
            (act_treatment_steel, -pole_tips_machining_scrap)
        ],
        database= motor_db,
        name_product= "machined pole tips"
    )

### Punching of electrical steel

Similar to the process of machining of the shaft, energy use is not available from the manufacturer so a similar process for the punching of electrical steel sheets described in Table 36 of Nordelöf et al. (2017a) is used. The quantity of scrap generated is provided by the manufacturer and differs for the rotor and stator.

In [57]:
punching_elec = 0.013 #kWh per kg of punched electrical steel
punching_rotor_efficiency = 0.36 #Ratio output/input
punching_stator_efficiency = 0.6 #Ratio output/input

In [58]:
def create_punching_rotor_lamination_activity(motor_db):
    punching_rotor_scrap = 1 / punching_rotor_efficiency - 1
    create_new_activity(
        act_name= "rotor lamination punching",
        unit= "kilogram",
        location = "SE",
        exchanges=[
            (act_elec_mv, punching_elec),
            (act_treatment_steel, -punching_rotor_scrap)
        ],
        database= motor_db,
        name_product= "punched rotor lamination"
    )

In [59]:
def create_punching_stator_lamination_activity(motor_db):
    punching_stator_scrap = 1 / punching_stator_efficiency - 1
    create_new_activity(
        act_name= "stator lamination punching",
        unit= "kilogram",
        location = "SE",
        exchanges=[
            (act_elec_mv, punching_elec),
            (act_treatment_steel, -punching_stator_scrap)
        ],
        database= motor_db,
        name_product= "punched stator lamination"
    )

### Rotor production

The steel shaft is machined into its specific shape. Electrical-steel sheets are punched and stacked around the shaft. Copper bars are inserted into slots in the stacked electrical-steel sheets and copper short-circuit rings are brazed on both sides of the copper bars to form a closed circuit. 
The inventory includes material inputs, including wasted amount of material for electrical steel and low-alloyed steel, and activities for the shaft machining and electrical steel sheet punching.

**As the composition of the rotor is confidential, the activity for rotor production is not provided in this document.**

### Stator impregnation

Activity to model the vacuum pressure impregnation (VPI) of the stator. The stator is dried in an oven. In a vacuum environment, the stator is immersed in a resin and the resin is pressed. Then the stator is baked until the resin is hardened, and rinsed.

Electricity use for vacuum pressing resin impregnation is based on the annual energy bill from 2020 of the building designated to this process divided by the number of machines produced. The quantity of resin used in the process and the quantity remaining in the stator is unknown. The resin not impregnated in the stator is reused so the amount of resin allocated to one motor would have been the amount in the final product. This incompleteness in the inventory is estimated to have little impact on the validity of the results. The inventory for resin impregnation is therefore limited to energy use.

In [60]:
impregnation_amount_elec = 3572.7 #kWh for one stator

In [61]:
def create_stator_impregnation_activity(motor_db):
    create_new_activity(
        act_name= "vacuum pressure impregnation of the stator",
        unit= "unit",
        location = "SE",
        exchanges=[
            (act_elec_mv, impregnation_amount_elec)
        ],
        database= motor_db,
        name_product= "stator impregnation"
    )

### Stator production

The stator is produced in three main steps:

- Production of the core: Electrical steel sheets are punched into the desired geometry. The sheets are then stacked and pressed. Steel bars are added and welded to a beginning and an end frame which hold together the laminates.
- Production of the coils: Copper wires are shaped into coils. The group of wires is insulated with insulation tape wrapping the coils.
- Assembly of the stator: The coils are taped to protect the insulation and ensure a good connection with the electrical steel. They are inserted in slots on the core. The coils are then coupled together and coupled to sockets. The stator is dried in an oven. In a vacuum environment, the stator is immersed in a resin and the resin is pressed. Then the stator is baked for the resin to harden and is rinsed. 

**As the composition of the stator is confidential, the activity for stator production is not provided in this document.**

### Bearings production

The inventory is limited to material requirements estimated from the bill of material and the material quantities ordered for manufacturing. Scraps are sent to recycling.

**As the composition of the bearings is confidential, the activity for bearings production is not provided in this document.**

### Frame and cooler production

The inventory is limited to material requirements estimated from the bill of material.

**As the composition of the frame and cooler is confidential, the activity for the frame and cooler production is not provided in this document.**

### Testing of the motor

The electricity used for the testing of one HV motor is based on the annual energy bill for the HV machine testing room over the year 2020 divided by the number of HV machines produced this year. The Coronavirus pandemic did not significantly impact the annual production volume. 

In [62]:
testing_amount_elec = 15971 #kWh per testing

In [63]:
def create_motor_testing_activity(motor_db):
    create_new_activity(
        act_name = "motor testing",
        unit = "unit",
        location = "SE",
        exchanges = [
            (act_elec_mv, testing_amount_elec)
        ],
        database = motor_db
    )

### Whole motor production

During the assembly phase, the rotor and stator are put together. Bearings are mounted on the shaft and a protective housing made of welded steel is added. Finally, the cooling system, made of welded steel, is added to the motor. The motor is then painted and tested.

**As the detailed composition of the motor is confidential, the detailed activity for motor production is not provided in this document. A simplified version is provided below as an example.**

In Jerome et al. (2023), the motor composition is provided as follow:

| Material            | Induction motor (ton) |
|---------------------|----------------------------|
| Steel, unalloyed    | 12.3                       |
| Steel, low-alloyed  | 4.1                        |
| Electrical steel    | 14.3                       |
| Copper, unalloyed   | 2.7                        |
| Copper, alloyed     | 0.9                        |
| Insulation (mica tape) | 0.4                        |

The simplified motor manufacturing activity does not include scraps and material production for scraped materials, and includes only the motor testing and stator impregnation as energy use during manufacturing. This is to avoid includin any confidential data.

In [64]:
amount_motor_steel = 12300 #kg
amount_motor_steel_low_alloyed = 4100 #kg
amount_motor_elec_steel = 14300 #kg
amount_motor_copper = 2700 #kg
amount_motor_brass = 900 #kg
amount_motor_insulation = 400 #kg
motor_weight = amount_motor_steel + amount_motor_steel_low_alloyed + amount_motor_elec_steel + amount_motor_copper + amount_motor_brass + amount_motor_insulation

In [65]:
def create_motor_production_activity(motor_db, motor_name):
    #Activities in input
    act_elec_steel = motor_db.search("electrical steel sheet production")[0]
    act_mica_tape = motor_db.search("mica tape")[0]
    act_impregnation = motor_db.search("vacuum pressure impregnation of the stator")[0]
    act_motor_testing = motor_db.search("motor testing")[0]
    #List of inputs
    list_exchanges = [
        (act_steel_unalloyed, amount_motor_steel),
        (act_steel_low_alloyed, amount_motor_steel_low_alloyed),
        (act_elec_steel, amount_motor_elec_steel),
        (act_copper, amount_motor_copper),
        (act_brass_market, amount_motor_brass),
        (act_mica_tape, amount_motor_insulation),
        (act_impregnation, 1),
        (act_motor_testing, 1)
    ]
    #Activity creation
    create_new_activity(
        act_name = motor_name + " motor production",
        unit = "unit",
        location = "SE",
        exchanges = list_exchanges,
        database = motor_db,
        name_product = motor_name + " motor"
    )

## Motor use

Energy consumption in the use phase consists of useful energy to be delivered by the motor, and of energy losses reflected by the energy efficiency value of the motor. As the purpose of the study in Jerome et al. (2023) is to compare motors delivering the same output and the production system fed by those motors is outside the scope, only the electricity due to energy losses is included. The energy efficiency can be estimated to remain constant during operation according to the manufacturer. Proper maintenance is assumed, involving the regular replacement of some small components such as filters, bolts and bearings. As the weight of those exchanged components is not significant in the bill of materials, additional component production related to maintenance is excluded. Transportation from the manufacturer and to the recycling facility is not included. It was identified as representing a negligible share of the environmental impact of HV motors (Westberg, 2021) and, in addition, the comparison of the motors does not include variations in geographical location for use.

In [66]:
def create_motor_use_activity(motor_db, motor, lifetime, operating_hours):
    amount_losses = motor["output"] * operating_hours * lifetime * (1 / motor["efficiency"] - 1)
    create_new_activity(
        act_name = motor["name"] + " motor use",
        unit = "unit",
        location = "SE",
        exchanges = [
            (act_elec_mv, amount_losses)
        ],
        database = motor_db,
        name_product = motor["name"] + " motor, used"
    )

In [67]:
def create_motor_use_per_year_activity(motor_db, motor, operating_hours):
    amount_losses = motor["output"] * operating_hours * (1 / motor["efficiency"] - 1)
    create_new_activity(
        act_name = motor["name"] + " motor use for one year",
        unit = "unit",
        location = "SE",
        exchanges = [
            (act_elec_mv, amount_losses)
        ],
        database = motor_db,
        name_product = motor["name"] + " motor, used for one year"
    )

## End-of-life treatment of the motors

HV motors are sent to recycling after use. The recycling process is similar to LV electric motors. This process is described in Tillman et al. (2020) and discussion with experts from the recycling company handling electric motors was used to point to differences between LV and HV motors.

For HV motors, motors are dismantled with heavy equipment to heat and cut the motors in order to avoid damages in hammer mills used for shredding. The components are then shredded, and the resulting pieces are sorted. The sorting starts with air separation, in order to separate the heavy fraction from the light fraction. The light fraction is processed to separate fluff, made of insulation, plastics, etc., to powdered metals such as copper. The light fraction is then sent to incineration. The heavy fraction mostly consists of different metals. Magnetic separation is performed to collect steel-rich fragments, which are sent to steel recycling. 
For LV motors, the remaining fraction consists mainly of “meatballs”, pieces of entangled copper, insulation and electrical steel. However, due to the size of HV motors being far superior to the size of the pieces after shredding and due to copper being a softer metal than steel, the generation of meatballs during the shredding is unlikely to happen for HV motors. After magnetic separation, it is assumed that all the steel is collected for recycling and that the remaining fraction is copper-rich fragments that are sent to copper recycling.

### Shredding and sorting

Electricity and heat are required during dismantling, shredding and sorting. Tillman et al. (2020) reports that the treatment of 1 kg of LV traction motors by shredding and sorting requires between 36 and 41Wh of electricity and between 0.015 and 0.019MJ of heat depending on the motor. Energy use for dismantling is unknown and depends on the extent of the dismantling. The highest values from Tillman et al. (2020) are used to minimise underestimation of the energy use for the motor treatment process.

In [68]:
shredding_amount_heat = 0.019 #MJ
shredding_amount_elec = 0.041 #kWh

In [69]:
def create_shredding_activity(motor_db):
    create_new_activity(
        act_name = "shredding and sorting processes for electric motors",
        unit = "kilogram",
        location = "SE",
        exchanges = [
            (act_heat, shredding_amount_heat),
            (act_elec_mv, shredding_amount_elec)
        ],
        database = motor_db,
        name_product = "motor, shredded and sorted"
    )

### Motor EoL treatment activity

This activity combines the shredding and sorting of the motor, the treatment of metal scraps and incineration of plastics.

The sorting rates for the different materials are based on the material flow analysis of LV traction motor treatment from Tillman et al. (2020). The rates are adapted to the material content of HV motors (no permanent magnet nor aluminium) and to the differences in treatment between LV and HV motors (no meatballs after shredding of HV motors).

In [73]:
sorting_rate_steel = 1
sorting_rate_copper = 0.9588
sorting_rate_plastic = 0

In [74]:
plastic_to_copper = 0.8069
plastic_to_steel = 0.0369
plastic_to_incineration = 1 - plastic_to_copper - plastic_to_steel

In [75]:
copper_to_steel = 0.0117
copper_to_incineration = 1 - sorting_rate_copper - copper_to_steel

In [86]:
def create_motor_eol_treatment_activity(motor_db, motor):
    #Useful amounts in motor
    amount_steel = amount_motor_steel + amount_motor_steel_low_alloyed + amount_motor_elec_steel
    amount_copper = amount_motor_copper + amount_motor_brass
    #Amounts to be treated
    copper_treatment = amount_copper * sorting_rate_copper + amount_motor_insulation * plastic_to_copper
    steel_treatment = amount_steel * sorting_rate_steel + amount_copper * copper_to_steel + amount_motor_insulation * plastic_to_steel
    #Useful activities
    act_motor_shredding = motor_db.search("shredding and sorting processes for electric motors")[0]
    #Activity
    create_new_activity(
        act_name = motor["name"] + " motor end-of-life treatment",
        unit = "unit",
        location = "SE",
        exchanges = [
            (act_motor_shredding, motor_weight),
            (act_incineration_copper, - amount_copper * copper_to_incineration),
            (act_treatment_copper, - copper_treatment),
            (act_treatment_steel, - steel_treatment),
            (act_incineration_plastic, - amount_motor_insulation * plastic_to_incineration)
        ],
        database = motor_db,
        name_product = motor["name"] + " motor, treated"
    )

## Repair

Stator rewinding is performed by pulling out and recycling the old copper windings, and by producing new windings that are inserted into the existing stator core. The windings are made of copper and insulation.

**As the composition of the windings is confidential, the detailed activity for repair by rewinding is not provided in this document. A mock activity is provided below.**

To avoid giving information on the motor composition, a mock activity for rewinding is given assuming that all the copper and all insulation in the motor are replaced.

In [77]:
def create_motor_rewinding_activity(motor_db, motor):
    #Amounts from the data table
    amount_copper = amount_motor_copper
    amount_insulation = amount_motor_insulation
    #Amounts to be treated
    copper_treatment = amount_copper * sorting_rate_copper + amount_insulation * plastic_to_copper
    #Activities in input
    act_mica_tape = motor_db.search("mica tape")[0]
    #Activity creation
    create_new_activity(
        act_name= motor["name"] + " rewinding",
        unit= "unit",
        location = "SE",
        exchanges=[
            (act_copper, amount_copper), #new windings
            (act_mica_tape, amount_insulation), #insulation in the new windings
            (act_treatment_copper, - copper_treatment), #treatment of copper from old windings
            (act_incineration_plastic, - amount_insulation * plastic_to_incineration) # treatment of insulation from old windings
        ],
        database= motor_db
    )

## Database creation

### List of motors to be included

The study focuses on the induction motor presented in Jerome et al. (2023). The characteristics of the motor are presented below.

In [78]:
induction_motor = {
    "name": "IM",
    "type": "induction",
    "efficiency": 0.973,
    "output": 15900 #kW
}

### Parameters

Except for two weeks of annual maintenance, the motors are assumed to run at full load 24h a day.

In [79]:
operating_hours = 50 * 7 * 24 #hours

Such big and stationary electric motors are typically used for at least 20 years.

In [80]:
lifetime = 20 #years

### Creation of the database with all motors

In [85]:
def create_motor_db(name_motor_db, list_motors, lifetime, operating_hours):
    reset_brightway_database(name_motor_db)
    motor_db = bd.Database(name_motor_db)
    motor_db.register()
    #Activities common to all motors
    create_silicon_seel_production_activity(motor_db)
    create_electrical_seel_production_activity(motor_db)
    create_mica_tape_production_activity(motor_db)
    create_shaft_machining_activity(motor_db)
    create_pole_tips_machining_activity(motor_db)
    create_punching_rotor_lamination_activity(motor_db)
    create_punching_stator_lamination_activity(motor_db)
    create_stator_impregnation_activity(motor_db)
    create_motor_testing_activity(motor_db)
    create_shredding_activity(motor_db)
    # Activities specific to different motors
    for motor in list_motors:
        create_motor_production_activity(motor_db, motor["name"])
        create_motor_use_activity(motor_db, motor, lifetime, operating_hours)
        create_motor_use_per_year_activity(motor_db, motor, operating_hours)
        create_motor_eol_treatment_activity(motor_db, motor)
        create_motor_rewinding_activity(motor_db, motor)

In [87]:
name_motor_db="motor_production_simplified"
list_motors = [
    induction_motor
]
create_motor_db(name_motor_db, list_motors, lifetime, operating_hours)

In [88]:
motor_db = bd.Database(name_motor_db)